In [2]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from tqdm import trange

In [ ]:
train_df['discourse_text'].values

In [ ]:
train_df = pd.read_csv("./input/train.csv")
tokenizer = Tokenizer()
texts = [x.lower() for x in train_df['discourse_text'].values]
tokenizer.fit_on_texts(texts)
total_words = len(tokenizer.word_index) + 1
# print(tokenizer.word_index)
# print(total_words)

In [39]:
input_sequences = []

for text in tqdm(texts):
    # print(text)
    token_list = tokenizer.texts_to_sequences(text)
    # print(token_list)
    for i in range(1,len(token_list)):
        ngram_sequences = token_list[:i+1]
        input_sequences.append(ngram_sequences)

max_sequences_len = max([len(x) for x in input_sequences])


        

100%|██████████| 36765/36765 [01:55<00:00, 319.38it/s]


In [5]:
import spacy

In [6]:
nlp = spacy.load("en_core_web_lg")

In [107]:
texts_tensor = []
for x in texts[:10]:
    texts_tensor.append(nlp(x).vector)

In [119]:
texts_tensor = tf.reshape(texts_tensor,[10,1,300])

In [110]:
texts_tensor

<tf.Tensor: shape=(1, 10, 300), dtype=float32, numpy=
array([[[-1.7310259 ,  1.2019125 , -3.6277354 , ...,  0.0363916 ,
         -2.994872  ,  1.4502984 ],
        [-0.8571301 ,  1.7046484 , -3.1711113 , ..., -0.6690958 ,
         -4.5095615 ,  1.7122338 ],
        [-0.7345443 ,  3.1622272 , -2.7153568 , ...,  0.02443947,
         -3.869576  ,  2.00546   ],
        ...,
        [-3.285236  ,  0.88648003, -2.619274  , ..., -0.62785   ,
         -2.2943168 ,  4.129184  ],
        [-1.8806963 ,  2.1315725 , -3.1053443 , ..., -0.08605026,
         -2.4896052 ,  2.1499102 ],
        [-2.3043728 ,  0.34056365, -2.421952  , ..., -1.7717329 ,
         -2.6810198 ,  0.9267128 ]]], dtype=float32)>

In [ ]:
for x in texts_tensor:
    print(x.shape)
    for y in x:
        print(y.shape)

In [121]:
model = tf.keras.Sequential()
# model.add(keras.layers.Embedding(total_words, 100, input_length=max_sequences_len-1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(300,return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
adam = tf.keras.optimizers.Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [65]:
layers = keras.layers.Bidirectional(keras.layers.LSTM(150))

In [20]:
labels = train_df['discourse_effectiveness']

In [122]:
def get_label(Keys,values,features):
    init = tf.lookup.KeyValueTensorInitializer(Keys,values)
    table = tf.lookup.StaticHashTable(init,default_value=-1)
    return table.lookup(tf.constant(features))

In [123]:
TARGET = tf.constant(['Adequate', 'Effective','Ineffective'])

TARGET_LABELS = tf.constant([0,1,2])
y_train = tf.one_hot(get_label( TARGET, TARGET_LABELS, labels[:10]),3)

In [124]:
y_train.shape

TensorShape([10, 3])

In [125]:
y_train = tf.reshape(y_train,[10,3])

In [ ]:

ys = tf.keras.utils.to_categorical(labels[:10], num_classes=3)

In [95]:
texts_tensor.shape

TensorShape([1, 200, 300])

In [126]:
# layers(tf.reshape(text_1,[1,1,300]))
model.fit(texts_tensor,y_train,epochs=10,verbose=1)

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 1.1155 - accuracy: 0.1000
Epoch 2/10
1/1 [==============================] - 0s 12ms/step - loss: 1.5110 - accuracy: 0.8000
Epoch 3/10
1/1 [==============================] - 0s 13ms/step - loss: 0.6300 - accuracy: 0.8000
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 1.0131 - accuracy: 0.8000
Epoch 5/10
1/1 [==============================] - 0s 12ms/step - loss: 0.8762 - accuracy: 0.8000
Epoch 6/10
1/1 [==============================] - 0s 12ms/step - loss: 0.6743 - accuracy: 0.8000
Epoch 7/10
1/1 [==============================] - 0s 13ms/step - loss: 0.5661 - accuracy: 0.8000
Epoch 8/10
1/1 [==============================] - 0s 12ms/step - loss: 0.5381 - accuracy: 0.8000
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 0.5233 - accuracy: 0.8000
Epoch 10/10
1/1 [==============================] - 0s 12ms/step - loss: 0.4924 - accuracy: 0.8000


In [67]:
y_train.shape

TensorShape([200, 3])

In [62]:
text_batches = tf.data.Dataset.from_tensor_slices(texts_tensor).batch(32)
y_batches = tf.data.Dataset.from_tensor_slices(y_train).batch(32)

In [63]:
data_set = tf.data.Dataset.zip( (text_batches , y_batches) )

In [ ]:
for x_batch, y_batch in data_set.batch(10):
    for x in x_batch:
        for y in x:
            print(y.shape)


In [207]:
inputs = tf.random.normal([32, 10, 8])

In [55]:
data_set

<ZipDataset element_spec=(TensorSpec(shape=(None, 2, 300), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>

In [211]:
inputs[0]

<tf.Tensor: shape=(10, 8), dtype=float32, numpy=
array([[-1.8753177 ,  0.39924157, -1.3061185 , -0.09128398, -1.8712283 ,
        -0.34483775, -0.57712895,  1.4799764 ],
       [ 0.11513742, -1.3900502 ,  0.20380816,  0.6482683 , -1.6172249 ,
         0.3066741 ,  0.07384012, -0.9795225 ],
       [ 0.87422425,  0.78189695,  0.01507624,  0.17949   , -1.6737931 ,
        -0.45730028, -0.81652325,  0.3563014 ],
       [-0.19982137,  0.84976166, -0.59731007, -0.96287096, -0.6522744 ,
        -0.16280177, -1.1096549 ,  1.2835126 ],
       [ 1.3445842 , -0.97756916, -0.31488225,  0.58931786, -0.13619326,
         1.5717092 ,  0.32090935,  0.80906075],
       [ 1.4670334 , -0.119016  , -0.06259945,  1.2847294 , -1.0686811 ,
        -0.51476103, -0.40006658,  0.66146886],
       [ 1.060258  ,  0.70658004,  0.45708534, -1.5078332 , -1.2828417 ,
        -0.70203847,  0.09950235, -1.3442135 ],
       [ 1.0623285 ,  0.75111175, -1.0949779 ,  0.32393587, -1.5980583 ,
        -1.4131682 ,  0.3217770